In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2 
import time
from scipy.sparse.linalg import svds

In [ ]:
# Load the data
cap = cv2.VideoCapture("highway.avi")
init_frame = 90 
cap.set(cv2.CAP_PROP_POS_FRAMES, init_frame)

num_frames = 30
frames = []
num_channels = 3
frame_num = 0

for i in range(num_frames):
    ret, frame = cap.read()
    if not ret:
        break
    frame_num += 1
    frames.append(frame)

cap.release()
num_frames = frame_num

In [ ]:
# Initialize variables
n, m, _ = frames[0].shape
 
# Flatten the frame
max_val = np.max(np.abs(frames)) 
D = np.zeros((n*m*num_channels, num_frames))
 
for i in range(num_frames):
    D[:, i] = frames[i].flatten() 
D = D/max_val  

# Add noise
np.random.seed(0)
noise_level = 0.0
D = D + noise_level*np.random.randn(D.shape[0], D.shape[1])

# Sanity check
print(D.shape)
print(np.max(D))

In [ ]:
U, S, V = np.linalg.svd(D, full_matrices=False)
print(np.dot(U, S))

In [ ]:

def algorithm_run(D, lamda, mu, error_tol = 1e-8, max_iter = 100, print_every = 10):
    L = np.zeros(D.shape) 
    S = np.zeros(D.shape) 
    Y = np.zeros(D.shape) 

    iter = 0    
    error = np.inf
    mu_inv = 1/mu

    # Solve for L and S
    while error > error_tol and iter < max_iter:
        # L update
        U, s, V = np.linalg.svd(D - S + mu_inv*Y, full_matrices=False)
        # Threshold singular values
        s = np.sign(s)*np.maximum(np.abs(s) - mu, 0)
        L = np.dot(U, np.dot(np.diag(s), V))

        # S update
        M = D - L + mu_inv*Y
        S = np.sign(M)*np.maximum(np.abs(M) - mu*lamda, 0)

        # Y update
        Y = Y + mu*(D - L - S)

        # Error
        error = np.linalg.norm(D - L - S, 'fro')/np.linalg.norm(D, 'fro')
        cvx_error = np.linalg.norm(L, 'nuc') + lamda*np.linalg.norm(S.flatten(), 1)
        iter += 1  
        if iter % print_every == 0:
            print(f"Iteration {iter}, error = {cvx_error}", \
                  f"L rank = {np.linalg.matrix_rank(L)}, S l1 = {np.linalg.norm(S.flatten(), 1)}")
    
    return L, S, cvx_error

In [ ]:

# Cross-validation ranges for hyper-parameters
lambdas = np.logspace(-3, 3, 7)
mus = np.logspace(-3, 1, 4) # [m*n/(4*np.linalg.norm(D, 1))]

# Hyper-parameter tuning
best_error = np.inf
best_lambda = 0
best_mu = 0
for lamda in lambdas:
    for mu in mus:
        # Measure the time to run the algorithm 
        t = time.time()  
        L, S, error = algorithm_run(D, lamda, mu, max_iter = 10)
        print(f"Time to run the algorithm = {time.time() - t}", f"Error = {error}",\
               f"L rank = {np.linalg.matrix_rank(L)}", f"S L1 = {np.linalg.norm(S.flatten(), 1)}") 
        if error < best_error:
            best_error = error
            best_lambda = lamda
            best_mu = mu

print(f"Best error = {best_error}, lambda = {best_lambda}, mu = {best_mu}")


In [ ]:
# Run the algorithm with the best hyper-parameters
print("Running the algorithm with the best hyper-parameters", best_lambda, best_mu)
L, S, error = algorithm_run(D, best_lambda, best_mu, error_tol = 1e-7, max_iter = 10, print_every = 1) 
print(f"Error = {error}", f"L rank = {np.linalg.matrix_rank(L)}", f"S L1 = {np.linalg.norm(S.flatten(), 1)}")